In [2]:
pip install itk

     |████████████████████████████████| 20.2MB 46.7MB/s 
     |████████████████████████████████| 70.1MB 52kB/s 
     |████████████████████████████████| 53.6MB 87kB/s 
     |████████████████████████████████| 15.0MB 341kB/s 
     |████████████████████████████████| 16.6MB 264kB/s 
     |████████████████████████████████| 94.7MB 1.2MB/s 


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import itk

file_name = "Meningioma.nrrd"
imgfinal = itk.imread(file_name)
arrayfinal = itk.GetArrayFromImage(imgfinal)
array2d = np.take(arrayfinal, 75,0)
plt.imshow(array2d, cmap = plt.cm.gray)
plt.title('Original GrayScale MRI')
plt.rcParams['figure.figsize'] = [40,10]
plt.show()


RuntimeError: ignored

In [ ]:
A = np.zeros([256,256])

In [ ]:
for i in range(256):
    for j in range(256):
        A[i][j] = array2d[i][j]

In [ ]:
#plotting the MRI from the new array we created.

plt.imshow(A ,cmap = plt.cm.gray)
plt.rcParams['figure.figsize'] = [10,20]
plt.title('Image A copy')
plt.show()

A.max()

In [ ]:
amax = A.max()
amin = A.min()
A = A - amin
print('amin now')
print(amin)
print('amax now')
print(amax)
#formula to make values from [amin, amax] --> [0, 255]
ssize = amax - amin
amax = A.max()
amin = A.min()

In [ ]:
# the lenght of each bin to be created for 
# mapping values in the space [0 .. 255]

bbin = ssize/256;
bbin

index = np.zeros([256])

for i in range(256):
        index[i] = bbin*(i+1)
        
index[255]


In [ ]:
F = np.zeros([256,256])
k = 0

for i in range(256):
    
    for j in range(256):
        run = 1
        k = 0
        less = 0
        great = 0
        print("Testing for [%d %d]" %(i,j))
        while(run == 1):
            
            #print("Checking: %f  <= %f" %(A[i][j], index[k]))

            if(A[i][j] <= index[k]):
                
                if(great == 1):
                    #print("Assign found")
                    F[i][j] = k;
                    #print("Assigned: %d" %(k))
                    run = 0
                    break
                    
                
                if(less == 1):
                    #print("Assign found")
                    F[i][j] = k-1;
                    #print("Assigned: %d" %(k-1))
                    run = 0
                    break
                    
                k = k+1
                less = 1
                #print("Less = 1")
            else:
                #print("Greater")
                great = 1
                k = k+1 
            if(k == 255):
                F[i][j] = 255
                break;
# created F array with values
        
        
F

In [ ]:
plt.imshow(F , cmap=plt.cm.gray)
plt.rcParams['figure.figsize'] = [10,20]
plt.title('Image with mapped values in 0-255')
plt.show()


In [ ]:
# creation of a colormap (PseudoColorTranslation)
# and also creation of 3 different new arrays.(R, G, B) based on the choosen colormap.
from colorsys import hsv_to_rgb

def pseudocolor(val, minval, maxval):
    
    h = (float(val-minval) / (maxval-minval)) * 255

    # Convert hsv color (h,1,1) to its rgb equivalent.
    # Note: hsv_to_rgb() function expects h to be in the range 0..1 not 0..360
    r, g, b = hsv_to_rgb(h/255, 1., 1.)
    return r, g, b



In [ ]:
pseudocolor(23,0,255)

In [ ]:
R = np.zeros([256,256])
G = np.zeros([256,256])
B = np.zeros([256,256])

for i in range(256):
    for j in range(256):
        R[i][j], G[i][j], B[i][j] = pseudocolor(F[i][j],0,255)

In [ ]:

# Images after the Pseudo Color Translation process.
plt.subplot(1,3,1)

plt.imshow(R,cmap = plt.cm.binary)
plt.title('R')
plt.subplot(1,3,2)
plt.imshow(G,cmap = plt.cm.binary)
plt.title('G')
plt.subplot(1,3,3)
plt.imshow(B,cmap = plt.cm.binary)
plt.title('B')
plt.rcParams['figure.figsize'] = [10,10]
plt.show()



In [ ]:
# Next process is the Color Space Translation (CST)
L = np.zeros([256,256]) #luminocity
a = np.zeros([256,256]) #a and
b = np.zeros([256,256]) #b according to paper.

import math

# diff function used in the calculation
# of a number's the cubic root.
def diff(n, mid) : 
    if (n > (mid * mid * mid)) : 
        return (n - (mid * mid * mid)) 
    else : 
        return ((mid * mid * mid) - n) 
          
# Returns cube root of a no n 
def cubicRoot(n) : 
      
    # Set start and end for binary  
    # search 
    start = 0
    end = n 
      
    times = 0
        
    # Set precision 
    e = 0.0000001
  
    while (True) : 
        #print('again') #dbg 
        
        times = times + 1
        
        mid = (start + end) / 2
        error = diff(n, mid) 
        
        if(times > 10):
            return mid
        # If error is less than e  
        # then mid is our answer 
        # so return mid 
        if (error == e) : 
            return mid
        if(error < e):
            return mid
              
        # If mid*mid*mid is greater 
        # than n set end = mid 
        if ((mid * mid * mid) > n) : 
            end = mid 
              
        # If mid*mid*mid is less  
        # than n set start = mid 
        else : 
            start = mid 

# H function as defined in the paper.
def h(x):
    
    if(x>0.008856):
        return cubicRoot(x)
    else:
        return ( (7.787*x) + 16/116)
    

In [ ]:
# Ys, Zs, Ws standard stimulus coefficients.
#D65 used as found in Wikipedia.
Ys = 100;
#Zs = 108.8840;
Zs = 82.5188; # D50
Ws = 95.0489;

for i in range(256):
#    print("row: %d" %(i)) #dbg
    
    for j in range(256):
        #print("col: %d" %(j)) #dbg
        W = 0.4304*R[i][j] + 0.3416*G[i][j] + 0.1784*B[i][j]
        Y = 0.2219*R[i][j] + 0.7068*G[i][j] + 0.0713*B[i][j]
        Z = 0.0202*R[i][j] + 0.1296*G[i][j] + 0.9393*B[i][j]
        L[i][j] = 116* h(Y/Ys) - 16;
        a[i][j] = 500*h(W/Ws) - h(Y/Ys)
        b[i][j] = 200*h(Y/Ys)- h(Z/Zs)       

In [ ]:
# visualization of L, a, b
plt.subplot(1,3,1)
plt.imshow(L,cmap = plt.cm.binary)
plt.title('L')
plt.subplot(1,3,2)
plt.imshow(a,cmap = plt.cm.binary)
plt.title('a')
plt.subplot(1,3,3)
plt.imshow(b,cmap = plt.cm.binary)
plt.title('b')
plt.rcParams['figure.figsize'] = [20,30]
plt.show()


In [ ]:
# Next process is the K-means clustering of the *a *b planes
K = np.zeros([256*256,2])
idx = 0

for i in range(256):
    for j in range(256):
        K[idx][0]= b[i][j]
        K[idx][1]= a[i][j]
        idx = idx + 1
#array K holds value to be fed in the k-means algorithm.

In [ ]:
from sklearn.cluster import KMeans
cl_n = 10
kmeans = KMeans(n_clusters= cl_n, random_state=0).fit(K)
# 3 clusters selected according to the authors in the proposed paper.
Labels = kmeans.labels_
Labels = Labels.reshape(256,256)
#plt.subplot(1,cl_n+1,1)

desimg = np.zeros([256,256])


for k in range(cl_n):
    blanktestimg = np.zeros([256,256])
    for i in range(256): 
        for j in range(256):
            if (Labels[i][j] == k) :
                blanktestimg[i][j] = 1
                if(k==9):
                    desimg[i][j] = F[i][j]
     
    plt.subplot(1,cl_n+1,k+1)
    plt.imshow(blanktestimg, cmap = plt.cm.gray)
    plt.title('Cluster %d' %(k))

plt.subplot(1,cl_n+1,cl_n+1)
plt.imshow(Labels,cmap = plt.cm.afmhot)
plt.rcParams['figure.figsize'] = [30,10]
plt.title('Clustering result')
plt.show()



In [ ]:
plt.imshow(desimg,cmap = plt.cm.gray)
plt.rcParams['figure.figsize'] = [30,10]
plt.title('Desired CLuster')
plt.show()

In [ ]:
# Next L array should be used for the process of histogram-based clustering.

# Rescale L in range[0,255 ?]
lmin = L.min()
lmax = L.max()
ldist = lmax - lmin

Lprocessed = L.copy()


if(lmin < 0):
    Lprocessed = Lprocessed + (-lmin)
else:
    Lprocessed = Lprocessed + (lmin)

space_len = Lprocessed.max() - Lprocessed.min()
diamerisi = space_len/256
index2 = np.zeros([256])

# creating the indexes for mapping 0...255 in Lprocessed array correctly.
for i in range(256):
        index2[i] = diamerisi*(i+1)
        #print(index2[i])


In [ ]:
for i in range(256):

    for j in range(256):
      
        #print( " %d %d " %(i,j))
        less = 0
        great = 0
        run = 1 # traverse the index2 array.
        k = 0
            
        while(run == 1):
            #print("check : %f  <= %f" %(Lprocessed[i][j], index2[k]))
            if( Lprocessed[i][j] <= index2[k] ):   
                
                if(great == 1):
                    Lprocessed[i][j] = k
                    run = 0
                    break
                    
                if(less == 1):
                    Lprocessed[i][j] = k-1
                    run = 0
                    break
                k = k + 1
                less = 1
            else:
                great = 1
                k = k + 1
                
            if(k == 255):
                Lprocessed[i][j] = 255
                break
                

In [ ]:
hist, bin_edges = np.histogram(Lprocessed, bins=256)
plt.plot(hist)
plt.title('L(Luminosity histogram)')
plt.rcParams['figure.figsize'] = [8,3]
plt.show()


In [ ]:
#what is the region we should choose? (for example region of interest??)
fim = np.zeros([256,256])

lim_low = 235
lim_high = 255

for i in range(256):
    for j in range(256):
        if(Lprocessed[i][j] > lim_low ):
            if(Lprocessed[i][j]< lim_high):
                fim[i][j] = desimg[i][j]
                #if(desimg[i][j] == 1):
                 #   fim[i][j] = 255
                #else:
                 #   fim[i][j] = 0


#segmentation based on pixel coordinates. 
fseg = np.zeros([256,256])

for i in range(256):
    for j in range(256):        
        if(i>60 and i<125 and j > 100  and j < 150):
            fseg[i][j] = fim[i][j]
                
plt.subplot(1,4,1)
plt.imshow(Labels ,cmap=plt.cm.coolwarm, interpolation='nearest')
plt.title("Classification Results")

plt.subplot(1,4,2)
plt.imshow(fim ,cmap=plt.cm.coolwarm, interpolation='nearest')
plt.title("Tumor area after histogram thresholds")

plt.subplot(1,4,3)
plt.imshow(fseg ,cmap=plt.cm.coolwarm, interpolation='nearest')
plt.title("Segmented tumor area")

plt.subplot(1,4,4)
plt.imshow(Lprocessed ,cmap=plt.cm.coolwarm, interpolation='nearest')
plt.title("Luminocity L mapped in [0-255]")



plt.rcParams['figure.figsize'] = [20,160]
plt.show()